In [104]:
import json
import re
from utils import collapse_annotations

Ignore these blocks of code here

In [105]:
imsitu_space = json.load(open('imsitu_space.json'))

In [106]:
def get_word_gender_map(wordsfile = "/juice/scr/rtaori/imsitu_data/words.txt"):
    word_map = {}
    wordmap2 = {}
    M = ['man']
    F = ['woman']
    with open(wordsfile) as words_f:
        for line in words_f:
            words = line.strip().split('\t')
            word_map[words[0]] = words[1]
            all_words = re.split(' |, ', words[1])
            gender = set()
            for m in M:
                if m in all_words:
                    gender.add('m')
                    break
            for f in F:
                if f in all_words:
                    gender.add('f')
                    break
            tmp = ''.join(list(gender))
            wordmap2[words[0]] = tmp
        wordmap2['null'] = 'null'
    return word_map, wordmap2

In [107]:
word_map, wordmap2 = get_word_gender_map()
male_nouns, female_nouns = [], []
for noun_id, gender in wordmap2.items():
    if gender == 'm':
        male_nouns.append(noun_id)
    if gender == 'f':
        female_nouns.append(noun_id)

This calculates the gender ratio for cooking on the training set

In [108]:
man_noun, woman_noun = 'n10287213', 'n10787470'
train_set = json.load(open('train.json'))

In [109]:
man_count, woman_count = 0, 0
for i, (image, info) in enumerate(train_set.items()):
    if info['verb'] == 'cooking':
        agents = [frame['agent'] for frame in info['frames']]
#         is_man = any(noun in agents for noun in male_nouns)
#         is_woman = any(noun in agents for noun in female_nouns)
        is_man = man_noun in agents
        is_woman = woman_noun in agents
        if is_man and not is_woman:
            man_count += 1
        if is_woman and not is_man:
            woman_count += 1
print(man_count, woman_count, woman_count / (man_count + woman_count))

33 68 0.6732673267326733


In [110]:
def get_gender_ratios(dataset_or_preds, only_use_main_noun=True):
    man_count, woman_count = 0, 0
    for i, (image, info_or_pred) in enumerate(dataset_or_preds.items()):
        info = info_or_pred[0] if type(info_or_pred) == list else info_or_pred
        if info['verb'] == 'cooking': # or True and 'agent' in info['frames'][0]:
            agents = [frame['agent'] for frame in info['frames']]
            if only_use_main_noun:
                is_man = man_noun in agents
                is_woman = woman_noun in agents
            else:
                is_man = any(noun in agents for noun in male_nouns)
                is_woman = any(noun in agents for noun in female_nouns)
            if is_man and not is_woman:
                man_count += 1
            if is_woman and not is_man:
                woman_count += 1
    return man_count, woman_count, woman_count / (man_count + woman_count)

This calculates the cooking gender ratio for model predictions and the dataset 

In [111]:
train_set = json.load(open('train.json'))
dev_set = json.load(open('dev.json'))
test_set = json.load(open('test.json'))
data_all = train_set | dev_set | test_set

In [112]:
get_gender_ratios(data_all)

(58, 110, 0.6547619047619048)

In [113]:
get_gender_ratios(train_set)

(33, 68, 0.6732673267326733)

In [114]:
get_gender_ratios(dev_set)

(13, 26, 0.6666666666666666)

In [115]:
sum(['cooking' in k for k, v in dev_set.items()])

50

In [116]:
len(dev_set)

25200

In [117]:
# import sys, importlib
# importlib.reload(sys.modules['utils'])
# from utils import collapse_annotations
collapsed_train_set = collapse_annotations(train_set, use_majority=True)
get_gender_ratios(collapsed_train_set)

(26, 59, 0.6941176470588235)

In [118]:
dev_preds = json.load(open('/juice/scr/rtaori/imsitu_feedback/baselinev2_preds.json'))
get_gender_ratios(dev_preds)

(8, 51, 0.864406779661017)

In [119]:
sum([v[0]['verb'] == 'cooking' for k, v in dev_preds.items()])

63

In [120]:
dev_preds = json.load(open('/juice/scr/rtaori/imsitu_feedback/resnet34_majority_v2_preds.json'))
get_gender_ratios(dev_preds)

(1, 11, 0.9166666666666666)

In [121]:
sum([v[0]['verb'] == 'cooking' for k, v in dev_preds.items()])

12

In [122]:
dev_preds = json.load(open('/juice/scr/rtaori/imsitu_feedback/resnet34_random_v2_preds.json'))
get_gender_ratios(dev_preds)

(3, 16, 0.8421052631578947)

In [123]:
sum([v[0]['verb'] == 'cooking' for k, v in dev_preds.items()])

20

In [124]:
dev_preds = json.load(open('/juice/scr/rtaori/imsitu_feedback/resnet34_baseline_preds.json'))
get_gender_ratios(dev_preds)

(5, 32, 0.8648648648648649)

In [125]:
sum([v[0]['verb'] == 'cooking' for k, v in dev_preds.items()])

41

In [126]:
for i, (image, info) in enumerate(train_set.items()):
    if info['verb'] == 'cooking':
        agents = [frame['agent'] for frame in info['frames']]
#         agents = [word_map[agent] if agent != '' else '' for agent in agents]
        agents = ['man' if agent in wordmap2 and wordmap2[agent] == 'm' else agent for agent in agents]
        agents = ['woman' if agent in wordmap2 and wordmap2[agent] == 'f' else agent for agent in agents]
#         agents = ['woman_noun' if agent == woman_noun else agent for agent in agents]
        print(i, agents)

110 ['n10084295', 'n10084295', 'n09917593']
122 ['n07942152', 'n00007846', 'n07942152']
192 ['woman', 'woman', 'woman']
205 ['man', 'n07942152', 'n07942152']
612 ['n10285313', 'n09917593', 'n10285313']
1543 ['man', 'man', 'man']
1619 ['man', 'man', 'n09963574']
2413 ['n10084295', 'woman', 'n10084295']
3011 ['woman', 'woman', 'woman']
3069 ['', 'n05217688', 'woman']
3376 ['man', 'woman', 'woman']
3428 ['woman', 'woman', 'woman']
3504 ['', '', 'n00007846']
3952 ['woman', 'woman', 'woman']
4083 ['woman', 'woman', 'woman']
5742 ['n00007846', '', 'n00007846']
5787 ['woman', 'woman', 'woman']
5802 ['woman', 'woman', 'woman']
6137 ['woman', '', 'n00007846']
6208 ['man', 'man', 'man']
6258 ['man', 'man', 'man']
6481 ['man', 'man', 'man']
6688 ['woman', 'n09917593', 'n10084295']
8374 ['woman', 'woman', 'woman']
8488 ['n07985628', 'man', 'n07942152']
8640 ['', 'n00007846', 'n00007846']
9813 ['woman', 'woman', 'woman']
10426 ['n07942152', 'n00007846', 'n07942152']
10641 ['man', 'man', 'man']
1166

In [127]:
for i, (image, info) in enumerate(collapsed_train_set.items()):
    if info['verb'] == 'cooking':
        agents = [frame['agent'] for frame in info['frames']]
#         agents = [word_map[agent] if agent != '' else '' for agent in agents]
        agents = ['man' if agent in wordmap2 and wordmap2[agent] == 'm' else agent for agent in agents]
        agents = ['woman' if agent in wordmap2 and wordmap2[agent] == 'f' else agent for agent in agents]
#         agents = ['woman_noun' if agent == woman_noun else agent for agent in agents]
        print(i, agents)

110 ['n10084295']
122 ['n07942152']
192 ['woman']
205 ['n07942152']
612 ['n10285313']
1543 ['man']
1619 ['man']
2413 ['n10084295']
3011 ['woman']
3069 ['n05217688']
3376 ['woman']
3428 ['woman']
3504 ['n00007846']
3952 ['woman']
4083 ['woman']
5742 ['n00007846']
5787 ['woman']
5802 ['woman']
6137 ['n00007846']
6208 ['man']
6258 ['man']
6481 ['man']
6688 ['n10084295']
8374 ['woman']
8488 ['n07985628']
8640 ['n00007846']
9813 ['woman']
10426 ['n07942152']
10641 ['man']
11666 ['woman']
11702 ['woman']
12130 ['n10285313']
12407 ['man']
13037 ['n07942152']
13597 ['woman']
13951 ['n00007846']
16146 ['n07942152']
16377 ['n09963574']
16520 ['woman']
16638 ['woman']
17421 ['woman']
17983 ['n10084295']
18079 ['n00007846']
19808 ['man']
20813 ['woman']
22771 ['woman']
24417 ['n10285313']
24525 ['man']
24538 ['woman']
24755 ['woman']
25037 ['woman']
25511 ['woman']
25682 ['n00007846']
25882 ['n10285313']
26025 ['man']
26169 ['woman']
28224 ['n00007846']
29946 ['woman']
30037 ['woman']
30201 ['woma

In [128]:
word_map['n10084295']

'female child, girl, little girl'